# 1.3 Basic SELECT Queries

This section introduces the fundamental SELECT statement - the foundation of data retrieval in SQL.

## Learning Objectives
By the end of this section, you will be able to:
- Write basic SELECT statements to retrieve data
- Select specific columns vs. all columns
- Use aliases to rename columns
- Apply LIMIT to control result size
- Understand query execution order

## Prerequisites
- Completed sections 1.1 and 1.2
- Understanding of table structure
- Sample data from previous sections

## Database Connection

Let's connect to our database and verify our sample data is ready.

In [1]:
import sqlite3
import pandas as pd
from IPython.display import display

# Connect to our database
conn = sqlite3.connect('my_database.db')
cursor = conn.cursor()

# Verify our tables exist
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cursor.fetchall()

print("✅ Database connection established!")
print(f"📚 Available tables: {[table[0] for table in tables]}")

✅ Database connection established!
📚 Available tables: ['sqlite_sequence', 'departments', 'employees', 'projects', 'customers']


## The SELECT Statement Structure

The basic syntax of a SELECT statement:

```sql
SELECT column1, column2, ...
FROM table_name;
```

### Key Components:
- **SELECT**: Specifies which columns to retrieve
- **FROM**: Specifies which table to query
- **;**: Ends the SQL statement

In [2]:
# Example 1: Select all columns with *
print("📊 Example 1: Select all columns from departments")
df = pd.read_sql_query("SELECT * FROM departments", conn)
display(df)

print("\n" + "="*50 + "\n")

# Example 2: Select specific columns
print("📊 Example 2: Select specific columns from employees")
df = pd.read_sql_query("SELECT first_name, last_name, email FROM employees", conn)
display(df)

print("\n" + "="*50 + "\n")

# Example 3: Select with calculated columns
print("📊 Example 3: Select with calculations")
df = pd.read_sql_query("""
SELECT 
    first_name, 
    last_name, 
    salary,
    salary * 12 as annual_salary
FROM employees
""", conn)
display(df)

📊 Example 1: Select all columns from departments


,dept_id,dept_name,location,budget,created_date,is_active
0,1,Human Resources,New York,500000,2020-01-01,1
1,2,Engineering,San Francisco,2000000,2019-06-15,1
2,3,Marketing,Chicago,750000,2020-03-01,1
3,4,Sales,Los Angeles,1200000,2019-12-01,1
4,5,Finance,Boston,600000,2020-02-15,1




📊 Example 2: Select specific columns from employees


,first_name,last_name,email
0,John,Doe,john.doe@company.com
1,Jane,Smith,jane.smith@company.com
2,Mike,Johnson,mike.johnson@company.com
3,Sarah,Williams,sarah.williams@company.com
4,David,Brown,david.brown@company.com




📊 Example 3: Select with calculations


,first_name,last_name,salary,annual_salary
0,John,Doe,75000,900000
1,Jane,Smith,82000,984000
2,Mike,Johnson,65000,780000
3,Sarah,Williams,90000,1080000
4,David,Brown,95000,1140000


## Column Aliases

Aliases allow you to rename columns in your query results for better readability.

### Alias Syntax:
- `column_name AS alias_name`
- `column_name alias_name` (AS is optional)

In [3]:
# Example: Using aliases for better column names
print("🏷️ Using Column Aliases:")

df = pd.read_sql_query("""
SELECT 
    dept_id AS "Department ID",
    dept_name AS "Department Name",
    location AS "Office Location",
    budget AS "Annual Budget",
    is_active AS "Currently Active"
FROM departments
""", conn)
display(df)

print("\n" + "="*30 + "\n")

# Example: Aliases with calculations
print("🧮 Aliases with Calculations:")
df = pd.read_sql_query("""
SELECT 
    project_name AS "Project",
    budget AS "Budget ($)",
    budget / 1000 AS "Budget (K$)",
    ROUND(budget / 1000, 1) AS "Budget (Rounded K$)"
FROM projects
""", conn)
display(df)

🏷️ Using Column Aliases:


,Department ID,Department Name,Office Location,Annual Budget,Currently Active
0,1,Human Resources,New York,500000,1
1,2,Engineering,San Francisco,2000000,1
2,3,Marketing,Chicago,750000,1
3,4,Sales,Los Angeles,1200000,1
4,5,Finance,Boston,600000,1




🧮 Aliases with Calculations:


,Project,Budget ($),Budget (K$),Budget (Rounded K$)
0,Website Redesign,150000,150,150.0
1,Mobile App Development,300000,300,300.0
2,Marketing Campaign Q2,75000,75,75.0
3,Sales Training Program,50000,50,50.0
4,Financial System Upgrade,200000,200,200.0


## Limiting Results with LIMIT

The LIMIT clause controls how many rows are returned in your result set.

### LIMIT Syntax:
- `LIMIT n` - Returns first n rows
- `LIMIT n OFFSET m` - Returns n rows starting from row m+1

In [4]:
# Example 1: Using LIMIT to get first 3 employees
print("🔢 Example 1: First 3 employees")
df = pd.read_sql_query("SELECT * FROM employees LIMIT 3", conn)
display(df)

print("\n" + "="*30 + "\n")

# Example 2: Using LIMIT with OFFSET
print("🔢 Example 2: Employees 2-4 (OFFSET demonstration)")
df = pd.read_sql_query("SELECT first_name, last_name, salary FROM employees LIMIT 3 OFFSET 1", conn)
display(df)

print("\n" + "="*30 + "\n")

# Example 3: Top 2 highest budget projects
print("🔢 Example 3: Projects ordered by budget (we'll learn ORDER BY in detail later)")
df = pd.read_sql_query("""
SELECT project_name, budget 
FROM projects 
ORDER BY budget DESC 
LIMIT 2
""", conn)
display(df)

🔢 Example 1: First 3 employees


,emp_id,first_name,last_name,email,hire_date,salary,dept_id
0,1,John,Doe,john.doe@company.com,2020-01-15,75000,2
1,2,Jane,Smith,jane.smith@company.com,2019-03-22,82000,2
2,3,Mike,Johnson,mike.johnson@company.com,2021-06-10,65000,3




🔢 Example 2: Employees 2-4 (OFFSET demonstration)


,first_name,last_name,salary
0,Jane,Smith,82000
1,Mike,Johnson,65000
2,Sarah,Williams,90000




🔢 Example 3: Projects ordered by budget (we'll learn ORDER BY in detail later)


,project_name,budget
0,Mobile App Development,300000
1,Financial System Upgrade,200000


## DISTINCT - Removing Duplicates

The DISTINCT keyword removes duplicate rows from your result set.

### DISTINCT Usage:
- `SELECT DISTINCT column` - Unique values from one column
- `SELECT DISTINCT col1, col2` - Unique combinations of multiple columns

In [5]:
# Example 1: Distinct departments (from employees table)
print("🎯 Example 1: Distinct department IDs from employees")
df = pd.read_sql_query("SELECT DISTINCT dept_id FROM employees", conn)
display(df)

print("\n" + "="*30 + "\n")

# Example 2: Distinct locations
print("🎯 Example 2: Distinct office locations")
df = pd.read_sql_query("SELECT DISTINCT location FROM departments", conn)
display(df)

print("\n" + "="*30 + "\n")

# Example 3: Distinct combinations
print("🎯 Example 3: Distinct status and priority combinations from projects")
df = pd.read_sql_query("SELECT DISTINCT status, priority FROM projects ORDER BY status, priority", conn)
display(df)

🎯 Example 1: Distinct department IDs from employees


,dept_id
0,2
1,3
2,1
3,4




🎯 Example 2: Distinct office locations


,location
0,New York
1,San Francisco
2,Chicago
3,Los Angeles
4,Boston




🎯 Example 3: Distinct status and priority combinations from projects


,status,priority
0,Active,3
1,Completed,2
2,In Progress,4
3,In Progress,5
4,Planning,5


## Combining Multiple Concepts

Let's combine what we've learned into more complex queries.

In [6]:
# Complex example combining all concepts
print("🚀 Complex Query Example:")
print("Get unique employee information with calculated annual salary, limited results")

df = pd.read_sql_query("""
SELECT DISTINCT
    first_name AS "First Name",
    last_name AS "Last Name", 
    email AS "Email Address",
    salary AS "Monthly Salary",
    salary * 12 AS "Annual Salary",
    dept_id AS "Department"
FROM employees
WHERE salary > 70000
ORDER BY salary DESC
LIMIT 3
""", conn)
display(df)

print("\n" + "="*30 + "\n")

# Another example with projects
print("🎯 Project Summary Example:")
df = pd.read_sql_query("""
SELECT 
    project_name AS "Project Name",
    status AS "Current Status",
    budget / 1000 AS "Budget (K$)",
    priority AS "Priority Level",
    CASE 
        WHEN priority >= 4 THEN 'High'
        WHEN priority = 3 THEN 'Medium' 
        ELSE 'Low'
    END AS "Priority Label"
FROM projects
ORDER BY priority DESC, budget DESC
LIMIT 4
""", conn)
display(df)

🚀 Complex Query Example:
Get unique employee information with calculated annual salary, limited results


,First Name,Last Name,Email Address,Monthly Salary,Annual Salary,Department
0,David,Brown,david.brown@company.com,95000,1140000,4
1,Sarah,Williams,sarah.williams@company.com,90000,1080000,1
2,Jane,Smith,jane.smith@company.com,82000,984000,2




🎯 Project Summary Example:


,Project Name,Current Status,Budget (K$),Priority Level,Priority Label
0,Mobile App Development,Planning,300,5,High
1,Financial System Upgrade,In Progress,200,5,High
2,Website Redesign,In Progress,150,4,High
3,Marketing Campaign Q2,Active,75,3,Medium


## Practice Exercises

Practice writing SELECT statements with the concepts you've learned:

1. **Basic Selection**: Get all information from the customers table
2. **Specific Columns**: Select only company names and emails from customers
3. **With Aliases**: Create meaningful column names for a departments query
4. **Limited Results**: Get the first 2 projects with their names and budgets
5. **Distinct Values**: Find all unique countries from the customers table

Complete the exercises below:

In [7]:
# Exercise 1: Basic selection from customers
print("Exercise 1: All customer information")
df = pd.read_sql_query("SELECT * FROM customers", conn)
display(df)

print("\n" + "="*30 + "\n")

# Exercise 2: Specific columns
print("Exercise 2: Company names and emails")
df = pd.read_sql_query("""
SELECT company_name, email 
FROM customers
""", conn)
display(df)

print("\n" + "="*30 + "\n")

# Exercise 3: Using aliases
print("Exercise 3: Departments with meaningful column names")
df = pd.read_sql_query("""
SELECT 
    dept_name AS "Department",
    location AS "Office Location", 
    budget AS "Annual Budget ($)",
    CASE 
        WHEN is_active = 1 THEN 'Active'
        ELSE 'Inactive'
    END AS "Status"
FROM departments
""", conn)
display(df)

print("\n" + "="*30 + "\n")

# Exercise 4: Limited results
print("Exercise 4: First 2 projects with names and budgets")
df = pd.read_sql_query("""
SELECT 
    project_name AS "Project Name",
    budget AS "Budget ($)"
FROM projects
LIMIT 2
""", conn)
display(df)

print("\n" + "="*30 + "\n")

# Exercise 5: Distinct values
print("Exercise 5: Unique countries from customers")
df = pd.read_sql_query("SELECT DISTINCT country FROM customers", conn)
display(df)

Exercise 1: All customer information


,customer_id,company_name,contact_name,email,phone,address,city,country,credit_limit
0,1,Tech Solutions Inc,Alice Johnson,alice@techsolutions.com,555-0101,123 Tech St,San Francisco,USA,50000
1,2,Global Marketing Ltd,Bob Smith,bob@globalmarketing.com,555-0102,456 Market Ave,New York,USA,75000
2,3,Innovation Corp,Carol Davis,carol@innovation.com,555-0103,789 Innovation Blvd,Austin,USA,100000




Exercise 2: Company names and emails


,company_name,email
0,Tech Solutions Inc,alice@techsolutions.com
1,Global Marketing Ltd,bob@globalmarketing.com
2,Innovation Corp,carol@innovation.com




Exercise 3: Departments with meaningful column names


,Department,Office Location,Annual Budget ($),Status
0,Human Resources,New York,500000,Active
1,Engineering,San Francisco,2000000,Active
2,Marketing,Chicago,750000,Active
3,Sales,Los Angeles,1200000,Active
4,Finance,Boston,600000,Active




Exercise 4: First 2 projects with names and budgets


,Project Name,Budget ($)
0,Website Redesign,150000
1,Mobile App Development,300000




Exercise 5: Unique countries from customers


,country
0,USA


## Query Performance Tips

### Best Practices for SELECT queries:
1. **Specify columns**: Use specific column names instead of `*` when possible
2. **Use LIMIT**: Add LIMIT to prevent accidentally large result sets
3. **Column order**: Put most selective columns first in multi-column operations
4. **Aliases**: Use meaningful aliases for calculated columns

### Understanding Query Execution:
1. **FROM** - Identify the source table
2. **SELECT** - Determine which columns to return  
3. **DISTINCT** - Remove duplicates if specified
4. **ORDER BY** - Sort results (covered in next section)
5. **LIMIT** - Restrict number of rows returned

In [8]:
# Demonstration of query performance considerations
print("📈 Query Performance Examples:")

# Less efficient - selecting all columns
print("❌ Less efficient (selecting all columns):")
df = pd.read_sql_query("SELECT * FROM employees", conn)
print(f"Columns returned: {len(df.columns)}")

print("\n" + "="*30 + "\n")

# More efficient - selecting only needed columns
print("✅ More efficient (selecting specific columns):")
df = pd.read_sql_query("SELECT first_name, last_name FROM employees", conn)
print(f"Columns returned: {len(df.columns)}")

print("\n📊 Memory usage difference is significant with larger datasets!")

📈 Query Performance Examples:
❌ Less efficient (selecting all columns):
Columns returned: 7


✅ More efficient (selecting specific columns):
Columns returned: 2

📊 Memory usage difference is significant with larger datasets!


## Section Summary

In this section, you mastered the fundamentals of data retrieval:

✅ **SELECT Statement**: The foundation of all data queries  
✅ **Column Selection**: Choosing specific columns vs. selecting all (*)  
✅ **Aliases**: Creating meaningful column names with AS  
✅ **LIMIT**: Controlling result set size for performance  
✅ **DISTINCT**: Removing duplicate rows from results  
✅ **Query Structure**: Understanding the logical flow of SELECT queries  

### Key SQL Commands:
- `SELECT column1, column2` - Specify columns to retrieve
- `SELECT *` - Retrieve all columns  
- `AS alias_name` - Create column aliases
- `LIMIT n` - Restrict number of rows
- `DISTINCT` - Remove duplicate rows
- `CASE WHEN` - Conditional logic in SELECT

### Query Structure Pattern:
```sql
SELECT [DISTINCT] column_list
FROM table_name  
[WHERE conditions]
[ORDER BY columns]
[LIMIT number];
```

### Next Steps
In section 1.4, you'll learn how to filter your data using WHERE clauses to find exactly the information you need.